In [0]:
from pyspark.sql.functions import explode_outer, explode
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType
session = SparkSession.builder.appName("Dynamic json").getOrCreate()
json = session.read.option("multiline", "true").json("/FileStore/tables/file1.json")


In [0]:
def explode_arrays(df):
    array_cols = []
    for c in df.dtypes:
        if c[1][:5] =="array":
            array_cols.append(c[0])
    for x in array_cols:
        df = df.withColumn(x,explode_outer(x))
    return df

def explode_structs(df):
    struct_cols = []
    x = session.createDataFrame([], StructType([]))
    parent=""
    for c in df.dtypes:
        if c[1][:6] =="struct":
            parent = c[0]
            struct_cols.append(c[0])
    
    for i in struct_cols:
        x = json.select(i + ".*")

    for i in x.columns:
        access_string = parent+"."+i
        new_col_name = i+"_"+parent
        df = df.withColumn(new_col_name, df[access_string])
    if len(parent)>0:
        df = df.drop(parent)

    return df


In [0]:
types =[]
for i in json.dtypes:
    if (i[1][:6] =="struct"): 
        types.append(i[1][:6]) 
    elif(i[1][:6] =="array<"):
        types.append(i[1][:5])
    else:
        pass 

In [0]:
while (len(types) > 0 ):
    json = explode_arrays(json)
    json = explode_structs(json)
    types =[]
    for i in json.dtypes:
        if (i[1][:6] =="struct"): 
            types.append(i[1][:6]) 
        elif(i[1][:6] =="array<"):
            types.append(i[1][:5])
        else:
            pass 


In [0]:
json.printSchema()

root
 |-- code: long (nullable = true)
 |-- message: string (nullable = true)
 |-- results_found: long (nullable = true)
 |-- results_shown: long (nullable = true)
 |-- results_start: string (nullable = true)
 |-- status: string (nullable = true)
 |-- apikey_restaurant_restaurants: string (nullable = true)
 |-- average_cost_for_two_restaurant_restaurants: long (nullable = true)
 |-- book_url_restaurant_restaurants: string (nullable = true)
 |-- cuisines_restaurant_restaurants: string (nullable = true)
 |-- currency_restaurant_restaurants: string (nullable = true)
 |-- deeplink_restaurant_restaurants: string (nullable = true)
 |-- establishment_types_restaurant_restaurants: string (nullable = true)
 |-- events_url_restaurant_restaurants: string (nullable = true)
 |-- featured_image_restaurant_restaurants: string (nullable = true)
 |-- has_online_delivery_restaurant_restaurants: long (nullable = true)
 |-- has_table_booking_restaurant_restaurants: long (nullable = true)
 |-- id_restauran